### Use `@traceable` / `traceable`

In [1]:
from openai import Client
from langsmith import traceable

openai = Client()

@traceable
def format_prompt(subject):
    return [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": f"What's a good name for a store that sells {subject}?"
        }
    ]

@traceable(run_type="llm")
def invoke_llm(messages):
    return openai.chat.completions.create(
        messages=messages, model="gpt-4o-mini", temperature=0
    )

@traceable
def parse_output(response):
    return response.choices[0].message.content

@traceable
def run_pipeline(subject: str):
    messages = format_prompt(subject)
    response = invoke_llm(messages)
    return parse_output(response)

In [2]:
run_pipeline("colorful socks")

'Here are some fun and catchy name ideas for a store that sells colorful socks:\n\n1. Sock It to Me\n2. Colorful Toes\n3. Happy Feet Boutique\n4. The Sock Spectrum\n5. Rainbow Socks Co.\n6. Funky Footwear\n7. Sock Parade\n8. Dazzle Socks\n9. Vibrant Soles\n10. The Sock Garden\n\nFeel free to mix and match or modify these suggestions to find the perfect name for your store!'

### Use the `trace` context manager (Python only)

In [5]:
import openai
import langsmith as ls
from langsmith.wrappers import wrap_openai

client = wrap_openai(openai.Client())

@ls.traceable(run_type='tool', name='Retrieve Context')
def my_tool(question: str) -> str:
    return "During this morning's meeting, we solved all world conflict."

def chat_pipeline(question: str):
    context = my_tool(question)
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context."},
        {"role": "user", "content": f"Question: {question}\nContext: {context}"}
    ]
    chat_completion = client.chat.completions.create(
        model="gpt-4o-mini", messages=messages
    )
    return chat_completion.choices[0].message.content

app_inputs = {"input": "Can you summarize this morning's meetings?"}

with ls.trace("Chat Pipeline", "chain", project_name="my_test", inputs=app_inputs) as rt:
    output = chat_pipeline("Can you summarize this morning's meetings?")
    rt.end(outputs={"output": output})

### Wrap the OpenAI client

In [6]:
import openai
from langsmith import traceable
from langsmith.wrappers import wrap_openai

client = wrap_openai(openai.Client())

@traceable(run_type="tool", name="Retrieve Context")
def my_tool(question: str) -> str:
    return "During this morning's meeting, we solved all world conflict."

@traceable(name="Chat Pipeline")
def chat_pipeline(question: str):
    context = my_tool(question)
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context."},
        {"role": "user", "content": f"Question: {question}\nContext: {context}"}
    ]
    chat_completion = client.chat.completions.create(
      model="gpt-4o-mini", messages=messages
    )
    return chat_completion.choices[0].message.content

chat_pipeline("Can you summarize this morning's meetings?")

"This morning's meeting was highly productive, as the attendees successfully addressed and resolved all world conflicts."

### Example usage

In [9]:
from typing import Any, Callable, Type, TypeVar

T = TypeVar('T')

def traceable_cls(cls: Type[T]) -> Type[T]:
    """Instrument all public methods in a class."""
    
    def wrap_method(name: str, method: Any) -> Any:
        if callable(method) and not name.startswith('__'):
            return traceable(name=f"{cls.__name__}.{name}")(method)
        return method

    # Handle __dict__ case
    for name in dir(cls):
        if not name.startswith("_"):
            try:
                method = getattr(cls, name)
                setattr(cls, name, wrap_method(name, method))
            except AttributeError:
                pass

    # Handle __slots__ case
    if hasattr(cls, "__slots__"):
        for slot in cls.__slots__:  # type: ignore[attr-defined]
            if not slot.startswith("__"):
                try:
                    method = getattr(cls, slot)
                    setattr(cls, slot, wrap_method(slot, method))
                except AttributeError:
                    # Skip slots that don't have a value yet
                    pass

    return cls

@traceable_cls
class MyClass:
    def __init__(self, some_val: int):
        self.some_val = some_val

    def combine(self, other_val: int):
        return self.some_val + other_val

MyClass(13).combine(29)

42